In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

In [2]:
class CNNModel(nn.Module):
    """docstring for CNNModel"""
    def __init__(self):
        super(CNNModel, self).__init__()
        # For valid padding -> padding = 0  and change sizes 
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=4, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()
        self.cnn2 = nn.Conv2d(in_channels=4, out_channels=8, kernel_size=5, stride=1, padding=2)
        self.relu2 = nn.ReLU()
        #Max/Avg  Pool 1
        self.pool1= nn.MaxPool2d(kernel_size=2)
        #self.pool1= nn.AvgPool2d(kernel_size=2)
        self.cnn3 = nn.Conv2d(in_channels=8 , out_channels=16, kernel_size=5,stride=1,padding=2)
        self.relu3 = nn.ReLU()
        self.cnn4 = nn.Conv2d(in_channels=16 , out_channels=32, kernel_size=5,stride=1,padding=2)
        self.relu4 = nn.ReLU()
        #Max/Avg Pool 2
        self.pool2= nn.MaxPool2d(kernel_size=2)
        #self.pool2= nn.AvgPool2d(kernel_size=2)
        #Fully connected 1
        self.fc1 = nn.Linear(32*7*7,1000)
        self.relu5 = nn.ReLU()
        self.fc2 = nn.Linear(1000,10)

    def forward(self,x):
        out = self.cnn1(x)
        out = self.relu1(out)
        out = self.cnn2(out)
        out = self.relu2(out)
        out = self.pool1(out)
        out = self.cnn3(out)
        out = self.relu3(out)
        out = self.cnn4(out)
        out = self.relu4(out)
        out = self.pool2(out)
        out = out.view(out.size(0),-1)
        out = self.fc1(out)
        out = self.relu5(out)
        out = self.fc2(out)
        return out

In [3]:
train_dataset = dsets.MNIST(root='./data',train=True,transform = transforms.ToTensor(),download=True)
test_dataset = dsets.MNIST(root='./data', train=False, transform = transforms.ToTensor())
batch_size = 128
n_iters = 4300
num_epochs = n_iters/(len(train_dataset)/batch_size)
num_epochs = int(num_epochs)

In [4]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False)

In [5]:
model = CNNModel()
if torch.cuda.is_available():
    model.cuda()
criterion = nn.CrossEntropyLoss()
learning_rate=0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [6]:
#Make load_model true if pre-trained model is present
load_model=True
if load_model is True:
    model.load_state_dict(torch.load('CNN_model.pkl'))

In [7]:
train_model=True
if train_model is True:
        iter = 0
        for epoch in range(num_epochs):
            for i,(images,labels) in enumerate(train_loader):
                if torch.cuda.is_available():
                    images=Variable(images.cuda())
                    labels= Variable(labels.cuda())
                else:
                    images=Variable(images)
                    labels= Variable(labels)
                optimizer.zero_grad()
                outputs=model(images)
                loss = criterion(outputs,labels)
                loss.backward()
                optimizer.step()
                iter+=1
                if iter%500==0:
                    print('Iteration = ',iter,' || Loss = ', loss.item())

Iteration =  500  || Loss =  2.298454761505127
Iteration =  1000  || Loss =  2.286841869354248
Iteration =  1500  || Loss =  0.2775621712207794
Iteration =  2000  || Loss =  0.211588054895401
Iteration =  2500  || Loss =  0.08199624717235565
Iteration =  3000  || Loss =  0.08795986324548721
Iteration =  3500  || Loss =  0.03779427707195282
Iteration =  4000  || Loss =  0.033981192857027054


In [8]:
correct=0
total=0
for images,labels in test_loader:
    if torch.cuda.is_available():
        images = Variable(images.cuda())
    else:
        images = Variable(images)
    outputs=model(images)
    _, predicted = torch.max(outputs.data,1)
    total = total + labels.size(0)
    if torch.cuda.is_available():
        correct = correct + (predicted.cpu()==labels.cpu()).sum()
    else:
        correct = correct + (predicted==labels).sum()
accuracy = 100 * float(correct)/float(total)
print('Iteration = ',iter,' || Loss = ', loss.item(),' || Test set Accuracy = ', accuracy)

Iteration =  4221  || Loss =  0.050962332636117935  || Accuracy =  98.12


In [9]:
save_model = True
if save_model is True:
    torch.save(model.state_dict(), 'CNN_model.pkl')